In [1]:
# Calculating the nearest shape for each record in MO python using spark and udf functions

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1608426013125_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# Spark Config
from pyspark import SparkConf
from pyspark import SparkContext

spark_conf = (SparkConf().set("spark.speculation", "false"))
sc = SparkContext.getOrCreate(conf = spark_conf)

# spark = sparkSession
hadoop_conf = spark._jsc.hadoopConfiguration()
hadoop_conf.set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
hadoop_conf.set("mapreduce.fileoutputcommitter.algorithm.version","2")

# installing required packages for this notebook session
sc.install_pypi_package("matplotlib")
sc.install_pypi_package("descartes")
sc.install_pypi_package("shapely")
sc.install_pypi_package("pyproj==2.2.2")
sc.install_pypi_package("pandas")
sc.install_pypi_package("geopy==1.20.0")
sc.install_pypi_package("haversine")

In [13]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
import geopy.distance

# Getting shapes
shapes_from_csv = spark.read.csv("s3://mobility-traces-sp/aux-files/gtfs/shapes.csv",header=True).toPandas()

# Broadcast shape variable
shapes = sc.broadcast(shapes_from_csv).value

# Find the nearest shape of each record in MO file
def get_distance(trace_x, trace_y, shape_id):
    
    candidate_shapes = shapes.loc[shapes['shape_id'] == shape_id]
    
    trace_coord = (trace_x,trace_y)
    
    min_distance = 999999999999
    
    min_shape_sequence = ""
    
    min_shape_coord_lat = 0
    
    min_shape_coord_lon = 0
    
    
    for _,shape in candidate_shapes.iterrows():
        shape_coord = (shape["shape_pt_lon"], shape["shape_pt_lat"])
        
#         distance = geopy.distance.distance(shape_coord,trace_coord).m
        distance = haversine_formula2(shape_coord,trace_coord)
        
        # if the distance is lower than the min_distance, so it is the new min_distance
        if distance <= min_distance:
            min_distance = distance
            min_shape_sequence = shape["shape_pt_sequence"]
            min_shape_coord_lat = shape["shape_pt_lat"]
            min_shape_coord_lon = shape["shape_pt_lon"]
            
    return (int(min_shape_sequence),min_distance,min_shape_coord_lat,min_shape_coord_lon)
    

schema = StructType([
    StructField("sequence", IntegerType()),
    StructField("distance", FloatType()),
    StructField("shape_lat", StringType()),
    StructField("shape_lon", StringType())
    
])

get_distance_udf = udf(get_distance, schema)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# https://stackoverflow.com/questions/47669895/how-to-add-multiple-columns-using-udf

for day in range(1,32):
    traces = spark.read.parquet("s3a://mobility-traces-sp/processed-data-avl-date/7-records-inside-sp-in-october-enriched-shapes/MO_1510"+str(day)+"/")
    traces.repartition(100)

    traces_with_shape = traces.withColumn("shape_info", get_distance_udf("longitude","latitude","shape_id")) \
                            .withColumn("shape_sequence",col("shape_info.sequence"))  \
                            .withColumn("shape_distance",col("shape_info.distance"))  \
                            .withColumn("shape_lat",col("shape_info.shape_lat"))  \
                            .withColumn("shape_lon",col("shape_info.shape_lon"))  \
                            .drop("shape_info")
    
    traces_with_shape.write.parquet("s3://mobility-traces-sp/processed-data-avl-date/8-map-matching/MO_1510"+str(day)+"/")


In [15]:
# Methods to calculate distances
# Helpful links
# https://github.com/ashutoshb418/Foodies-Visualization/blob/master/Foodies_Chain.ipynb
# https://janakiev.com/blog/gps-points-distance-python/
# https://www.kite.com/python/answers/how-to-find-the-distance-between-two-lat-long-coordinates-in-python
# https://towardsdatascience.com/calculating-distance-between-two-geolocations-in-python-26ad3afe287b

import math
from geopy import distance
import math

# -23.49150063078416, -46.65645272614161 # Rio de Janeiro
# -22.91593838910498, -43.184773075383035 # Masp museum in Sao Paulo

lat1 = -23.49150063078416
lon1 = -46.65645272614161
lat2 = -22.91593838910498
lon2 = -43.184773075383035
coord_1 = (lon1,lat1)
coord_2 = (lon2,lat2)

############### Geopy lib #######################
# distance_geopy = distance.distance(coord_1, coord_2).m
# print('distance using geopy Vincenty’s formula: ', distance_geopy)
    
distance_geopy_great_circle = distance.great_circle(coord_1, coord_2).m
print('distance using geopy great circle: ', distance_geopy_great_circle)

print("Geopy geodesic",distance.geodesic(coord_1, coord_2).meters,"meters")

print("Geopy Vincenty",distance.vincenty(coord_1, coord_2).meters,"meters")

############ Haversine Lib #######################
import haversine as hs

print("Haversine lib",hs.haversine(coord_1,coord_2,unit=hs.Unit.METERS))

############### Haversine formula ################

def haversine_formula(coord1, coord2):
    
    R = 6372800  # Earth radius in meters
    lat1, lon1 = float(coord1[0]),float(coord1[1])
    lat2, lon2 = float(coord2[0]),float(coord2[1])
    
    phi1, phi2 = math.radians(lat1), math.radians(lat2) 
    dphi       = math.radians(lat2 - lat1)
    dlambda    = math.radians(lon2 - lon1)
    
    a = math.sin(dphi/2)**2 + \
        math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2
    
    return 2*R*math.atan2(math.sqrt(a), math.sqrt(1 - a))


print("Harversine formula",haversine_formula(coord_1,coord_2),"meters")

def haversine_formula2(coord1,coord2):
    
    lon1, lat1 = coord1
    lon2, lat2 = coord2
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)
    
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    R = 6372800
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c
    return distance

print("Haversine formula2",haversine_formula2(coord_1,coord_2),"meters")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

distance using geopy great circle:  388681.3196170513
Geopy geodesic 388472.46938148164 meters
Geopy Vincenty 388472.46938007645 meters
Haversine lib 388681.3074154888
Harversine formula 388790.58460842626 meters
Haversine formula2 360624.01682591485 meters

In [ ]:
# Results according to research
# The Haversine is simple, but it does not have accuracy like Vincenty and Geodesic
# The Vicenty is more accurate
# The Geodesic is more accurate than Vincenty, but it has the lowest speed